<a href="https://colab.research.google.com/github/GarganoNicolas/EVDVai_DataEngTP1/blob/main/EDVai_1_DataEng.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 0. Importar Librerias y data 🐣

---



![](https://i.insider.com/5784fbfedd089520678b465d?width=700&format=jpeg&auto=webp)

In [282]:
import pandas as pd
import numpy as np
import re

In [283]:
path = 'https://raw.githubusercontent.com/GarganoNicolas/EVDVai_DataEngTP1/main/Utimas_Desvinculaciones.xlsx'

In [284]:
xls = pd.ExcelFile(path)

sheet_names = xls.sheet_names
print(sheet_names)

['Row data', 'Rangos', 'Managers']


In [285]:
dfs = {}

for sheet_name in sheet_names:
    dfs[sheet_name] = xls.parse(sheet_name)

data = dfs['Row data']
df_Rangos = dfs['Rangos']
df_Managers = dfs['Managers']

In [286]:
data.sample(5)

,Id,Nombre,Apellido,Area,Fecha comienzo,Fecha fin,Nivel,Grupo recruitment,Tiempo recruitment,Rango Salarial,RS competencia,Manager,Realizo Cursos
20,5,Franco,Rodríguez,Marketing,2017-05-01,2017-11-30,N5,Grupo C,6,Rango 5,Rango 4,1,SI
6,17,Guillermo,Ramírez,IT,2020-01-06,2022-05-31,N2,Grupo A,3,Rango 2,Rango 2,7,NO
4,13,Alejandro,Romero,IT,2020-06-01,2022-02-28,N3,Grupo A,3,Rango 4,Rango 3,7,SI
28,29,Dante,Martini,IT,2021-03-01,2021-12-31,N4,Grupo C,6,Rango 3,Rango 3,5,NO
24,16,Vanina,Benítez,IT,2019-08-05,2020-04-30,N5,Grupo C,5,Rango 6,Rango 6,5,SI


In [287]:
df_Rangos

,Rango,desde,hasta
0,Rango 1,150000,200000
1,Rango 2,200001,250000
2,Rango 3,250001,300000
3,Rango 4,300001,350000
4,Rango 5,350001,400000
5,Rango 6,400001,500000
6,Rango 7,500001,600000
7,Rango 8,600001,700000
8,Rango 9,700001,800000
9,Rango 10,800001,1000000


In [288]:
df_Managers

,Id,Nombre,Apellido
0,1,Rolando,Casas
1,2,Marina,Pitana
2,3,José,Feligres
3,4,Miriam,Pisani
4,5,Alejandro,Pitorino
5,6,Sonia,Kamlasky
6,7,Martín,Viglia
7,8,Ricardo,Ronaldo
8,9,Vanina,Quintana
9,10,Daniel,Quiroga


# 1. Join data en una tabla para hacer comparaciones 🦄

![](https://thirdeyetraveller.com/wp-content/uploads/Duke-of-Humfreys-Library-Harry-Potter-14-728x546.jpg)

In [289]:
df_Rangos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Rango   10 non-null     object
 1   desde   10 non-null     int64 
 2   hasta   10 non-null     int64 
dtypes: int64(2), object(1)
memory usage: 368.0+ bytes


## 1.1 Crear la columna pesitos (de modo simpatico) para integrar un una sola columna el rango 💰

In [290]:
df_Rangos[['desde', 'hasta']] = df_Rangos[['desde', 'hasta']].astype(str)
df_Rangos['pesitos'] = df_Rangos[['desde', 'hasta']].agg(' - '.join, axis=1)
df_Rangos

,Rango,desde,hasta,pesitos
0,Rango 1,150000,200000,150000 - 200000
1,Rango 2,200001,250000,200001 - 250000
2,Rango 3,250001,300000,250001 - 300000
3,Rango 4,300001,350000,300001 - 350000
4,Rango 5,350001,400000,350001 - 400000
5,Rango 6,400001,500000,400001 - 500000
6,Rango 7,500001,600000,500001 - 600000
7,Rango 8,600001,700000,600001 - 700000
8,Rango 9,700001,800000,700001 - 800000
9,Rango 10,800001,1000000,800001 - 1000000


## 1.2 Integrar la nueva columna y la Rangos df a la tabla de data 💇


In [291]:
# Renombrar columna para hacer un merge
df_Rangos.rename(columns={'Rango': 'Rango Salarial'}, inplace= True)
# Hacer merge
data = data.merge(df_Rangos[['Rango Salarial', 'pesitos']], on='Rango Salarial', how='left')
pesitos = data[['pesitos', 'Rango Salarial']].sort_values('Rango Salarial').drop_duplicates()
pesitos

,pesitos,Rango Salarial
29,150000 - 200000,Rango 1
25,200001 - 250000,Rango 2
2,250001 - 300000,Rango 3
1,300001 - 350000,Rango 4
20,350001 - 400000,Rango 5
10,400001 - 500000,Rango 6


In [292]:
data.sample(3)

,Id,Nombre,Apellido,Area,Fecha comienzo,Fecha fin,Nivel,Grupo recruitment,Tiempo recruitment,Rango Salarial,RS competencia,Manager,Realizo Cursos,pesitos
20,5,Franco,Rodríguez,Marketing,2017-05-01,2017-11-30,N5,Grupo C,6,Rango 5,Rango 4,1,SI,350001 - 400000
2,8,Francesca,López,Compras,2017-01-02,2021-06-30,N3,Grupo A,4,Rango 3,Rango 3,2,SI,250001 - 300000
27,23,Camila,Prieto,IT,2022-01-03,2022-07-01,N6,Grupo C,6,Rango 6,Rango 6,5,NO,400001 - 500000


## 1.3 Como estaba trabajando en Rango, queria saber si era relevante la columna, y comparar con rango competencia.
### Comentario: Como solo 2 personas se fueron por mayor rango y 3 por menor rango (!important) pero para ser de 30 casos y contemplando que los 3 que se fueron por menor rango no hay un insight relevante. Mi idea original era crear una columna que refleje estos datos


In [293]:
Rango_salarial = data[(data['Rango Salarial'] != data['RS competencia'])]
Rango_salarial[['Id', 'Rango Salarial', 'RS competencia', 'Area']]

,Id,Rango Salarial,RS competencia,Area
4,13,Rango 4,Rango 3,IT
7,19,Rango 4,Rango 5,Auditoria
18,26,Rango 2,Rango 3,Producción
20,5,Rango 5,Rango 4,Marketing
26,21,Rango 6,Rango 5,Compras


# 2. Unir datos de manager a tabla general 🏰


![](https://cdn.totallythebomb.com/wp-content/uploads/2016/10/Hogwarts-teachers.jpg)

In [294]:
df_Managers.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   Id        10 non-null     int64 
 1   Nombre    10 non-null     object
 2   Apellido  10 non-null     object
dtypes: int64(1), object(2)
memory usage: 368.0+ bytes


In [295]:
# Lo mismo que con Rango, pero ahora con dos columnas, Nomnre del Manager y Apellido del Manager
df_Managers.rename(columns={'Id': 'Manager', 'Nombre': 'Nombre del Manager', 'Apellido': 'Apellido del Manager'}, inplace=True)
data = data.merge(df_Managers[['Manager', 'Nombre del Manager', 'Apellido del Manager']], on='Manager', how='left')
data.sample(3)

,Id,Nombre,Apellido,Area,Fecha comienzo,Fecha fin,Nivel,Grupo recruitment,Tiempo recruitment,Rango Salarial,RS competencia,Manager,Realizo Cursos,pesitos,Nombre del Manager,Apellido del Manager
24,16,Vanina,Benítez,IT,2019-08-05,2020-04-30,N5,Grupo C,5,Rango 6,Rango 6,5,SI,400001 - 500000,Alejandro,Pitorino
22,12,Luis,Sánchez,Ventas,2020-07-13,2020-12-31,N5,Grupo C,5,Rango 6,Rango 6,8,NO,400001 - 500000,Ricardo,Ronaldo
28,29,Dante,Martini,IT,2021-03-01,2021-12-31,N4,Grupo C,6,Rango 3,Rango 3,5,NO,250001 - 300000,Alejandro,Pitorino


## 2.2 Ya que estaba trabajando con los manager queria saber si algun manager se dio de baja, lo cual no sucedio, bravo! Tanto tiempo trabajando juntos nos une como familia 💑

In [296]:
# Quiero saber si se fue algun manager...
apellido = data[(data['Nombre'] == data['Nombre del Manager'])]
apellido[['Nombre', 'Apellido', 'Nombre del Manager', 'Apellido del Manager']]

,Nombre,Apellido,Nombre del Manager,Apellido del Manager


# 3. Para analizar estas tres columnas: 'Area', 'Manager', 'Grupo recruitment', tome los promedios de 'Tiempo recruitment', 'Rango Salarial', 'Nivel', 'Meses_trabajados' ademas de la cantidad de bajas. Esto me dio como resultado una tabla para cada columna ⏰


![](https://media1.popsugar-assets.com/files/thumbor/0pWTZ5wKX0bH3ng2GVdurerhoV8/fit-in/1024x1024/filters:format_auto-!!-:strip_icc-!!-/2016/01/14/936/n/1922283/64212f264eda8026_snape-severus-snape-31324685-500-211/i/When-He-Gives-Some-Champion-Level-Side-Eye.gif)

In [297]:
def remove_non_numeric(s):
    return re.sub('[^0-9]+', '', s)

# Assuming 'data' is your DataFrame and it's already defined
data['Mes_comienzo'] = data['Fecha comienzo'].dt.to_period('M')
data['Mes_fin'] = data['Fecha fin'].dt.to_period('M')
data['Meses_trabajados'] = data['Mes_fin'] - data['Mes_comienzo']
data['Meses_trabajados'] = data['Meses_trabajados'].apply(lambda x: x.n)

mean_values = data[['Area', 'Manager', 'Grupo recruitment', 'Tiempo recruitment', 'Rango Salarial', 'Nivel', 'Meses_trabajados']]
mean_values[['Rango Salarial', 'Nivel']] = mean_values[['Rango Salarial', 'Nivel']].apply(lambda x: x.apply(remove_non_numeric))
mean_values[['Rango Salarial', 'Nivel']] = mean_values[['Rango Salarial', 'Nivel']].apply(pd.to_numeric, errors='coerce')



# List of column names to group by
group_columns = ['Area', 'Manager', 'Grupo recruitment']

# Dictionary to store the results for each group
group_results = {}

for column in group_columns:
    # Grouping and calculating mean values
    mean_values_group = mean_values.groupby(column)[['Tiempo recruitment', 'Rango Salarial', 'Nivel', 'Meses_trabajados']].mean()
    mean_values_group.reset_index(inplace=True)
    mean_values_group.columns = [column, 'Prom Tiempo recruitment', 'Prom Rango Salarial', 'Prom Nivel', 'Prom Meses_trabajados']

    # Merging the mean values back to the original DataFrame
    merged_data = mean_values.merge(mean_values_group, on=column, how='left')

    value_counts = data.groupby(column)[column].transform('count')
    merged_data[f'{column}_bajas'] = value_counts

    # Dropping unnecessary columns and duplicates
    merged_data = merged_data.drop(['Tiempo recruitment', 'Rango Salarial', 'Nivel', 'Meses_trabajados'], axis=1)


    # Storing the result in the dictionary
    if column == 'Manager':
        group_results[column] = merged_data[[column, 'Area', 'Prom Tiempo recruitment', 'Prom Rango Salarial', 'Prom Nivel', f'{column}_bajas', 'Prom Meses_trabajados']].drop_duplicates().sort_values(column).reset_index(drop=True)
    else:
        group_results[column] = merged_data[[column, 'Prom Tiempo recruitment', 'Prom Rango Salarial', 'Prom Nivel', f'{column}_bajas', 'Prom Meses_trabajados']].drop_duplicates().sort_values(column).reset_index(drop=True)






# Print the results
for column, result in group_results.items():
    print(f'\n\n promedios por {column}')
    display(result)





 promedios por Area


<ipython-input-297-6f0a6796b2db>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mean_values[['Rango Salarial', 'Nivel']] = mean_values[['Rango Salarial', 'Nivel']].apply(lambda x: x.apply(remove_non_numeric))
<ipython-input-297-6f0a6796b2db>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mean_values[['Rango Salarial', 'Nivel']] = mean_values[['Rango Salarial', 'Nivel']].apply(pd.to_numeric, errors='coerce')


,Area,Prom Tiempo recruitment,Prom Rango Salarial,Prom Nivel,Area_bajas,Prom Meses_trabajados
0,Auditoria,3.5,2.500000,2.500000,2,22.000000
1,Compras,3.8,3.400000,3.400000,5,43.600000
2,IT,4.5,3.875000,3.750000,8,16.875000
3,Legales,4.0,2.000000,2.000000,1,36.000000
4,Marketing,4.5,4.000000,4.000000,4,20.000000
5,Producción,4.0,4.000000,4.500000,4,34.750000
6,Ventas,4.5,4.166667,4.166667,6,11.833333




 promedios por Manager


,Manager,Area,Prom Tiempo recruitment,Prom Rango Salarial,Prom Nivel,Manager_bajas,Prom Meses_trabajados
0,1,Marketing,4.5,4.000000,4.000000,2,7.500000
1,2,Compras,3.8,3.400000,3.400000,5,43.600000
2,3,Legales,4.0,2.000000,2.000000,1,36.000000
3,4,Producción,4.5,4.000000,4.000000,2,35.000000
4,5,IT,5.4,4.400000,4.200000,5,8.600000
5,6,Producción,3.5,4.000000,5.000000,2,34.500000
6,7,IT,3.0,3.000000,3.000000,3,30.666667
7,8,Ventas,4.5,4.166667,4.166667,6,11.833333
8,9,Marketing,4.5,4.000000,4.000000,2,32.500000
9,10,Auditoria,3.5,2.500000,2.500000,2,22.000000




 promedios por Grupo recruitment


,Grupo recruitment,Prom Tiempo recruitment,Prom Rango Salarial,Prom Nivel,Grupo recruitment_bajas,Prom Meses_trabajados
0,Grupo A,3.727273,3.636364,3.727273,11,32.636364
1,Grupo B,3.625000,3.375000,3.750000,8,30.375000
2,Grupo C,5.181818,4.090909,3.818182,11,11.000000


## 3.1 Queiro indagar en los manager que tienen menor promedio de meses trabajados en su equipo 🎣

### 3.1.1 Manager 5 asumo que es un equipo de IT de desarrollo, ya que tienen mas nivel y mas rango salarial que manager 7 tambien de IT, lo que puede ser mantenimiento o soporte. Es decir otra area de IT y por ende otro manager
####Comentario: Ademas se diferencias en grupo de recultamiento, Grupo C se toma mas tiempo, pero trae empleados de mas nivel, pero duran menos. Supongo tendra que ver con tareas mas especificas, y una vez culminado el proyecto u ordenada la tarea principal por la cual vinieron buscan otros rumbos de acuerdo a su especialidad.

In [298]:
manager5 = data[data['Manager'] == 5]
manager5

,Id,Nombre,Apellido,Area,Fecha comienzo,Fecha fin,Nivel,Grupo recruitment,Tiempo recruitment,Rango Salarial,RS competencia,Manager,Realizo Cursos,pesitos,Nombre del Manager,Apellido del Manager,Mes_comienzo,Mes_fin,Meses_trabajados
19,3,Juan,Albornoz,IT,2018-06-04,2019-04-30,N4,Grupo C,4,Rango 4,Rango 4,5,NO,300001 - 350000,Alejandro,Pitorino,2018-06,2019-04,10
21,9,Nicolas,Martínez,IT,2021-01-04,2021-11-30,N2,Grupo C,6,Rango 3,Rango 3,5,SI,250001 - 300000,Alejandro,Pitorino,2021-01,2021-11,10
24,16,Vanina,Benítez,IT,2019-08-05,2020-04-30,N5,Grupo C,5,Rango 6,Rango 6,5,SI,400001 - 500000,Alejandro,Pitorino,2019-08,2020-04,8
27,23,Camila,Prieto,IT,2022-01-03,2022-07-01,N6,Grupo C,6,Rango 6,Rango 6,5,NO,400001 - 500000,Alejandro,Pitorino,2022-01,2022-07,6
28,29,Dante,Martini,IT,2021-03-01,2021-12-31,N4,Grupo C,6,Rango 3,Rango 3,5,NO,250001 - 300000,Alejandro,Pitorino,2021-03,2021-12,9


In [299]:
manager7 = data[data['Manager'] == 7]
manager7

,Id,Nombre,Apellido,Area,Fecha comienzo,Fecha fin,Nivel,Grupo recruitment,Tiempo recruitment,Rango Salarial,RS competencia,Manager,Realizo Cursos,pesitos,Nombre del Manager,Apellido del Manager,Mes_comienzo,Mes_fin,Meses_trabajados
3,10,Ana,Diaz,IT,2017-03-06,2020-11-30,N4,Grupo A,3,Rango 3,Rango 3,7,NO,250001 - 300000,Martín,Viglia,2017-03,2020-11,44
4,13,Alejandro,Romero,IT,2020-06-01,2022-02-28,N3,Grupo A,3,Rango 4,Rango 3,7,SI,300001 - 350000,Martín,Viglia,2020-06,2022-02,20
6,17,Guillermo,Ramírez,IT,2020-01-06,2022-05-31,N2,Grupo A,3,Rango 2,Rango 2,7,NO,200001 - 250000,Martín,Viglia,2020-01,2022-05,28


### 3.1.2 La gente de ventas tambien parece rotar basatnte, seria bueno indagar las razones

In [300]:
manager8 = data[data['Manager'] == 8]
manager8.sort_values('Fecha fin')

,Id,Nombre,Apellido,Area,Fecha comienzo,Fecha fin,Nivel,Grupo recruitment,Tiempo recruitment,Rango Salarial,RS competencia,Manager,Realizo Cursos,pesitos,Nombre del Manager,Apellido del Manager,Mes_comienzo,Mes_fin,Meses_trabajados
23,14,Raul,García,Ventas,2019-02-04,2020-03-31,N3,Grupo C,6,Rango 3,Rango 3,8,SI,250001 - 300000,Ricardo,Ronaldo,2019-02,2020-03,13
22,12,Luis,Sánchez,Ventas,2020-07-13,2020-12-31,N5,Grupo C,5,Rango 6,Rango 6,8,NO,400001 - 500000,Ricardo,Ronaldo,2020-07,2020-12,5
1,7,Anabella,Fernández,Ventas,2020-07-06,2021-03-31,N4,Grupo A,3,Rango 4,Rango 4,8,NO,300001 - 350000,Ricardo,Ronaldo,2020-07,2021-03,8
16,22,Matias,Bosques,Ventas,2018-12-03,2021-06-30,N4,Grupo B,3,Rango 4,Rango 4,8,SI,300001 - 350000,Ricardo,Ronaldo,2018-12,2021-06,30
17,25,Carolina,Petersen,Ventas,2021-07-05,2022-02-28,N7,Grupo B,4,Rango 6,Rango 6,8,SI,400001 - 500000,Ricardo,Ronaldo,2021-07,2022-02,7
25,20,Marcelo,Flores,Ventas,2021-09-06,2022-05-31,N2,Grupo C,6,Rango 2,Rango 2,8,SI,200001 - 250000,Ricardo,Ronaldo,2021-09,2022-05,8


### 3.1.3 En el area de marketing hubo dos ingresos que no prosperaron parece

In [301]:
manager1 = data[data['Manager'] == 1]
manager1.sort_values('Fecha fin')

,Id,Nombre,Apellido,Area,Fecha comienzo,Fecha fin,Nivel,Grupo recruitment,Tiempo recruitment,Rango Salarial,RS competencia,Manager,Realizo Cursos,pesitos,Nombre del Manager,Apellido del Manager,Mes_comienzo,Mes_fin,Meses_trabajados
20,5,Franco,Rodríguez,Marketing,2017-05-01,2017-11-30,N5,Grupo C,6,Rango 5,Rango 4,1,SI,350001 - 400000,Rolando,Casas,2017-05,2017-11,6
11,1,Romina,Brito,Marketing,2018-03-05,2018-12-31,N3,Grupo B,3,Rango 3,Rango 3,1,SI,250001 - 300000,Rolando,Casas,2018-03,2018-12,9


# 4. Quiero comparar las fechas de inicio y fin, para saber si se contrataron equipos, y si tiene que ver los periodos de ingresos con los periodos de baja. Cuantas personas entraron o se fueron a la vez
### Comentario: La mayor cantidad de desvinculaciones se dieron post comienzo de la pandemia. Seria bueno comparar con la cantidad de vinculaciones que se dieron en ese tiempo pero aun siguen trabajando 📅


![](https://static0.gamerantimages.com/wordpress/wp-content/uploads/2023/09/restricted-section.jpg?q=50&fit=contain&w=1140&h=&dpr=1.5_)

In [302]:
breve_filas_duplicadas_comienzo = data[data['Mes_comienzo'].duplicated(keep=False)].drop(['Nombre', 'Apellido', 'Area', 'Fecha comienzo', 'Fecha fin',
       'Nivel', 'Grupo recruitment', 'Tiempo recruitment', 'Rango Salarial',
       'RS competencia', 'Manager', 'Realizo Cursos', 'pesitos',
       'Nombre del Manager', 'Apellido del Manager', 'Mes_fin',
       ], axis=1).sort_values(['Mes_comienzo', 'Meses_trabajados']).set_index('Id')
breve_filas_duplicadas_fin = data[data['Mes_fin'].duplicated(keep=False)].sort_values('Mes_fin').drop(['Nombre', 'Apellido', 'Area', 'Fecha comienzo', 'Fecha fin',
       'Nivel', 'Grupo recruitment', 'Tiempo recruitment', 'Rango Salarial',
       'RS competencia', 'Manager', 'Realizo Cursos', 'pesitos',
       'Nombre del Manager', 'Apellido del Manager', 'Mes_comienzo',
       ], axis=1).sort_values(['Mes_fin', 'Meses_trabajados']).set_index('Id')
filas_duplicadas_comienzo = data[data['Mes_comienzo'].duplicated(keep=False)].drop(['Id',
        'Nombre', 'Apellido', 'pesitos', 'Fecha comienzo', 'Fecha fin',
        'Nombre del Manager', 'Apellido del Manager'], axis=1).sort_values(['Mes_comienzo', 'Area', 'Nivel', 'Rango Salarial'])
filas_duplicadas_fin = data[data['Mes_fin'].duplicated(keep=False)].sort_values('Mes_fin').drop(['Id',
        'Nombre', 'Apellido', 'pesitos', 'Fecha comienzo', 'Fecha fin',
        'Nombre del Manager', 'Apellido del Manager'], axis=1).sort_values(['Mes_comienzo', 'Area', 'Nivel', 'Rango Salarial'])
print('Comenzaron el mismo mes')
display(breve_filas_duplicadas_comienzo)
display(filas_duplicadas_comienzo)
print('Ultimaron el mismo mes')
display(breve_filas_duplicadas_fin)
display(filas_duplicadas_fin)


Comenzaron el mismo mes


,Mes_comienzo,Meses_trabajados
Id,,
10,2017-03,44
2,2017-03,57
4,2018-05,36
26,2018-05,40
17,2020-01,28
28,2020-01,29
12,2020-07,5
7,2020-07,8
20,2021-09,8


,Area,Nivel,Grupo recruitment,Tiempo recruitment,Rango Salarial,RS competencia,Manager,Realizo Cursos,Mes_comienzo,Mes_fin,Meses_trabajados
12,Compras,N3,Grupo B,3,Rango 2,Rango 2,2,SI,2017-03,2021-12,57
3,IT,N4,Grupo A,3,Rango 3,Rango 3,7,NO,2017-03,2020-11,44
0,Legales,N2,Grupo A,4,Rango 2,Rango 2,3,SI,2018-05,2021-05,36
18,Producción,N3,Grupo B,3,Rango 2,Rango 3,6,SI,2018-05,2021-09,40
6,IT,N2,Grupo A,3,Rango 2,Rango 2,7,NO,2020-01,2022-05,28
10,Producción,N7,Grupo A,4,Rango 6,Rango 6,6,SI,2020-01,2022-06,29
1,Ventas,N4,Grupo A,3,Rango 4,Rango 4,8,NO,2020-07,2021-03,8
22,Ventas,N5,Grupo C,5,Rango 6,Rango 6,8,NO,2020-07,2020-12,5
29,Auditoria,N1,Grupo C,3,Rango 1,Rango 1,10,SI,2021-09,2022-06,9
25,Ventas,N2,Grupo C,6,Rango 2,Rango 2,8,SI,2021-09,2022-05,8


Ultimaron el mismo mes


,Mes_fin,Meses_trabajados
Id,,
12,2020-12,5
21,2020-12,37
22,2021-06,30
8,2021-06,53
9,2021-11,10
15,2021-11,37
29,2021-12,9
24,2021-12,35
2,2021-12,57


,Area,Nivel,Grupo recruitment,Tiempo recruitment,Rango Salarial,RS competencia,Manager,Realizo Cursos,Mes_comienzo,Mes_fin,Meses_trabajados
2,Compras,N3,Grupo A,4,Rango 3,Rango 3,2,SI,2017-01,2021-06,53
12,Compras,N3,Grupo B,3,Rango 2,Rango 2,2,SI,2017-03,2021-12,57
26,Compras,N5,Grupo C,4,Rango 6,Rango 5,2,SI,2017-11,2020-12,37
5,Compras,N4,Grupo A,4,Rango 4,Rango 4,2,NO,2018-10,2021-11,37
16,Ventas,N4,Grupo B,3,Rango 4,Rango 4,8,SI,2018-12,2021-06,30
8,Producción,N6,Grupo A,5,Rango 6,Rango 6,4,SI,2019-01,2021-12,35
9,Compras,N2,Grupo A,4,Rango 2,Rango 2,2,SI,2019-06,2022-04,34
14,Marketing,N5,Grupo B,5,Rango 5,Rango 5,9,SI,2019-07,2022-04,33
6,IT,N2,Grupo A,3,Rango 2,Rango 2,7,NO,2020-01,2022-05,28
10,Producción,N7,Grupo A,4,Rango 6,Rango 6,6,SI,2020-01,2022-06,29


### 4.1.1 Una tabla interactiva para comparar columnas

In [303]:
from google.colab import data_table
data1 = data[['Mes_fin', 'Mes_comienzo', 'Meses_trabajados', 'Nivel', 'Area', 'Manager', 'Grupo recruitment']].sort_values('Mes_fin')
data_table.DataTable(data1)


,Mes_fin,Mes_comienzo,Meses_trabajados,Nivel,Area,Manager,Grupo recruitment
20,2017-11,2017-05,6,N5,Marketing,1,Grupo C
11,2018-12,2018-03,9,N3,Marketing,1,Grupo B
19,2019-04,2018-06,10,N4,IT,5,Grupo C
15,2019-12,2017-04,32,N3,Marketing,9,Grupo B
23,2020-03,2019-02,13,N3,Ventas,8,Grupo C
24,2020-04,2019-08,8,N5,IT,5,Grupo C
3,2020-11,2017-03,44,N4,IT,7,Grupo A
22,2020-12,2020-07,5,N5,Ventas,8,Grupo C
26,2020-12,2017-11,37,N5,Compras,2,Grupo C
7,2021-01,2018-02,35,N4,Auditoria,10,Grupo A


In [304]:
data_table.disable_dataframe_formatter()


# 5. Quiero ver los casos por duracion, los que menos meses trabajaron

![](https://media.harrypotterfanzone.com/voldemort-with-hands-by-his-head-1050x0-c-default.jpg)

In [305]:
menos_de_8_meses = data[data['Meses_trabajados'] < 8]
menos_de_8_meses

,Id,Nombre,Apellido,Area,Fecha comienzo,Fecha fin,Nivel,Grupo recruitment,Tiempo recruitment,Rango Salarial,RS competencia,Manager,Realizo Cursos,pesitos,Nombre del Manager,Apellido del Manager,Mes_comienzo,Mes_fin,Meses_trabajados
17,25,Carolina,Petersen,Ventas,2021-07-05,2022-02-28,N7,Grupo B,4,Rango 6,Rango 6,8,SI,400001 - 500000,Ricardo,Ronaldo,2021-07,2022-02,7
20,5,Franco,Rodríguez,Marketing,2017-05-01,2017-11-30,N5,Grupo C,6,Rango 5,Rango 4,1,SI,350001 - 400000,Rolando,Casas,2017-05,2017-11,6
22,12,Luis,Sánchez,Ventas,2020-07-13,2020-12-31,N5,Grupo C,5,Rango 6,Rango 6,8,NO,400001 - 500000,Ricardo,Ronaldo,2020-07,2020-12,5
27,23,Camila,Prieto,IT,2022-01-03,2022-07-01,N6,Grupo C,6,Rango 6,Rango 6,5,NO,400001 - 500000,Alejandro,Pitorino,2022-01,2022-07,6


In [306]:
solo_8_meses = data[data['Meses_trabajados'] == 8]
solo_8_meses

,Id,Nombre,Apellido,Area,Fecha comienzo,Fecha fin,Nivel,Grupo recruitment,Tiempo recruitment,Rango Salarial,RS competencia,Manager,Realizo Cursos,pesitos,Nombre del Manager,Apellido del Manager,Mes_comienzo,Mes_fin,Meses_trabajados
1,7,Anabella,Fernández,Ventas,2020-07-06,2021-03-31,N4,Grupo A,3,Rango 4,Rango 4,8,NO,300001 - 350000,Ricardo,Ronaldo,2020-07,2021-03,8
24,16,Vanina,Benítez,IT,2019-08-05,2020-04-30,N5,Grupo C,5,Rango 6,Rango 6,5,SI,400001 - 500000,Alejandro,Pitorino,2019-08,2020-04,8
25,20,Marcelo,Flores,Ventas,2021-09-06,2022-05-31,N2,Grupo C,6,Rango 2,Rango 2,8,SI,200001 - 250000,Ricardo,Ronaldo,2021-09,2022-05,8


In [307]:
menos_de_12_meses_mas_de_8 = data.query('Meses_trabajados > 8 & Meses_trabajados <= 12')
menos_de_12_meses_mas_de_8

,Id,Nombre,Apellido,Area,Fecha comienzo,Fecha fin,Nivel,Grupo recruitment,Tiempo recruitment,Rango Salarial,RS competencia,Manager,Realizo Cursos,pesitos,Nombre del Manager,Apellido del Manager,Mes_comienzo,Mes_fin,Meses_trabajados
11,1,Romina,Brito,Marketing,2018-03-05,2018-12-31,N3,Grupo B,3,Rango 3,Rango 3,1,SI,250001 - 300000,Rolando,Casas,2018-03,2018-12,9
19,3,Juan,Albornoz,IT,2018-06-04,2019-04-30,N4,Grupo C,4,Rango 4,Rango 4,5,NO,300001 - 350000,Alejandro,Pitorino,2018-06,2019-04,10
21,9,Nicolas,Martínez,IT,2021-01-04,2021-11-30,N2,Grupo C,6,Rango 3,Rango 3,5,SI,250001 - 300000,Alejandro,Pitorino,2021-01,2021-11,10
28,29,Dante,Martini,IT,2021-03-01,2021-12-31,N4,Grupo C,6,Rango 3,Rango 3,5,NO,250001 - 300000,Alejandro,Pitorino,2021-03,2021-12,9
29,30,Diego,Buzos,Auditoria,2021-09-06,2022-06-30,N1,Grupo C,3,Rango 1,Rango 1,10,SI,150000 - 200000,Daniel,Quiroga,2021-09,2022-06,9


In [308]:
menos_de_24_meses = data.query('Meses_trabajados > 12 & Meses_trabajados <= 24')
menos_de_24_meses

,Id,Nombre,Apellido,Area,Fecha comienzo,Fecha fin,Nivel,Grupo recruitment,Tiempo recruitment,Rango Salarial,RS competencia,Manager,Realizo Cursos,pesitos,Nombre del Manager,Apellido del Manager,Mes_comienzo,Mes_fin,Meses_trabajados
4,13,Alejandro,Romero,IT,2020-06-01,2022-02-28,N3,Grupo A,3,Rango 4,Rango 3,7,SI,300001 - 350000,Martín,Viglia,2020-06,2022-02,20
23,14,Raul,García,Ventas,2019-02-04,2020-03-31,N3,Grupo C,6,Rango 3,Rango 3,8,SI,250001 - 300000,Ricardo,Ronaldo,2019-02,2020-03,13


In [309]:
menos_de_36_meses = data.query('Meses_trabajados > 24 & Meses_trabajados <= 36')
menos_de_36_meses

,Id,Nombre,Apellido,Area,Fecha comienzo,Fecha fin,Nivel,Grupo recruitment,Tiempo recruitment,Rango Salarial,RS competencia,Manager,Realizo Cursos,pesitos,Nombre del Manager,Apellido del Manager,Mes_comienzo,Mes_fin,Meses_trabajados
0,4,Sebastian,González,Legales,2018-05-07,2021-05-30,N2,Grupo A,4,Rango 2,Rango 2,3,SI,200001 - 250000,José,Feligres,2018-05,2021-05,36
6,17,Guillermo,Ramírez,IT,2020-01-06,2022-05-31,N2,Grupo A,3,Rango 2,Rango 2,7,NO,200001 - 250000,Martín,Viglia,2020-01,2022-05,28
7,19,Rocio,Ruiz,Auditoria,2018-02-05,2021-01-30,N4,Grupo A,4,Rango 4,Rango 5,10,NO,300001 - 350000,Daniel,Quiroga,2018-02,2021-01,35
8,24,Micaela,Pauls,Producción,2019-01-07,2021-12-31,N6,Grupo A,5,Rango 6,Rango 6,4,SI,400001 - 500000,Miriam,Pisani,2019-01,2021-12,35
9,27,Rodrigo,Molinas,Compras,2019-06-03,2022-04-30,N2,Grupo A,4,Rango 2,Rango 2,2,SI,200001 - 250000,Marina,Pitana,2019-06,2022-04,34
10,28,Martina,Oliviera,Producción,2020-01-06,2022-06-30,N7,Grupo A,4,Rango 6,Rango 6,6,SI,400001 - 500000,Sonia,Kamlasky,2020-01,2022-06,29
13,6,Catalina,Gómez,Producción,2019-04-01,2022-03-31,N2,Grupo B,4,Rango 2,Rango 2,4,NO,200001 - 250000,Miriam,Pisani,2019-04,2022-03,35
14,11,Roberto,Pérez,Marketing,2019-07-01,2022-04-30,N5,Grupo B,5,Rango 5,Rango 5,9,SI,350001 - 400000,Vanina,Quintana,2019-07,2022-04,33
15,18,Pedro,Torres,Marketing,2017-04-03,2019-12-31,N3,Grupo B,4,Rango 3,Rango 3,9,SI,250001 - 300000,Vanina,Quintana,2017-04,2019-12,32
16,22,Matias,Bosques,Ventas,2018-12-03,2021-06-30,N4,Grupo B,3,Rango 4,Rango 4,8,SI,300001 - 350000,Ricardo,Ronaldo,2018-12,2021-06,30


In [310]:
mas_de_36_meses = data.query('Meses_trabajados >= 36')
mas_de_36_meses

,Id,Nombre,Apellido,Area,Fecha comienzo,Fecha fin,Nivel,Grupo recruitment,Tiempo recruitment,Rango Salarial,RS competencia,Manager,Realizo Cursos,pesitos,Nombre del Manager,Apellido del Manager,Mes_comienzo,Mes_fin,Meses_trabajados
0,4,Sebastian,González,Legales,2018-05-07,2021-05-30,N2,Grupo A,4,Rango 2,Rango 2,3,SI,200001 - 250000,José,Feligres,2018-05,2021-05,36
2,8,Francesca,López,Compras,2017-01-02,2021-06-30,N3,Grupo A,4,Rango 3,Rango 3,2,SI,250001 - 300000,Marina,Pitana,2017-01,2021-06,53
3,10,Ana,Diaz,IT,2017-03-06,2020-11-30,N4,Grupo A,3,Rango 3,Rango 3,7,NO,250001 - 300000,Martín,Viglia,2017-03,2020-11,44
5,15,Analia,Sosa,Compras,2018-10-01,2021-11-30,N4,Grupo A,4,Rango 4,Rango 4,2,NO,300001 - 350000,Marina,Pitana,2018-10,2021-11,37
12,2,Pedro,Lujan,Compras,2017-03-06,2021-12-31,N3,Grupo B,3,Rango 2,Rango 2,2,SI,200001 - 250000,Marina,Pitana,2017-03,2021-12,57
18,26,Fabian,Quiroz,Producción,2018-05-07,2021-09-30,N3,Grupo B,3,Rango 2,Rango 3,6,SI,200001 - 250000,Sonia,Kamlasky,2018-05,2021-09,40
26,21,Cristian,Martin,Compras,2017-11-06,2020-12-31,N5,Grupo C,4,Rango 6,Rango 5,2,SI,400001 - 500000,Marina,Pitana,2017-11,2020-12,37


# 6. A partir de aca es una cuenta pendiente que me quedo del pasado. Necesitaba cerrarla
## Mi proposito era armar un calendario que de cuenta del mes en que entro y salio
## Tambien queria graficarlo, pero eso quedara pendiente para el proximo curso imagino

![](https://www.noblecollection.com/ItemImages/Large/PRP%20HP%207017.jpg)



In [311]:
data['Fecha comienzo'] = pd.to_datetime(data['Fecha comienzo'])
data['Fecha fin'] = pd.to_datetime(data['Fecha fin'])
print(
    'Fecha comienzo:',
    ' min ',
    data['Fecha comienzo'].min(),
    ' max ',
    data['Fecha comienzo'].max(),
    '\nFecha fin:',
    ' min ',
    data['Fecha fin'].min(),
    ' max ',
    data['Fecha fin'].max()
    )

Fecha comienzo:  min  2017-01-02 00:00:00  max  2022-01-03 00:00:00 
Fecha fin:  min  2017-11-30 00:00:00  max  2022-07-01 00:00:00


In [312]:
start_date = data['Fecha comienzo'].min()
end_date = data['Fecha fin'].max()

date_range = pd.date_range(start=start_date, end=end_date, freq='M')
calendario = pd.DataFrame(date_range, columns=['Fecha']).T
calendario

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65
Fecha,2017-01-31,2017-02-28,2017-03-31,2017-04-30,2017-05-31,2017-06-30,2017-07-31,2017-08-31,2017-09-30,2017-10-31,2017-11-30,2017-12-31,2018-01-31,2018-02-28,2018-03-31,2018-04-30,2018-05-31,2018-06-30,2018-07-31,2018-08-31,2018-09-30,2018-10-31,2018-11-30,2018-12-31,2019-01-31,2019-02-28,2019-03-31,2019-04-30,2019-05-31,2019-06-30,2019-07-31,2019-08-31,2019-09-30,2019-10-31,2019-11-30,2019-12-31,2020-01-31,2020-02-29,2020-03-31,2020-04-30,2020-05-31,2020-06-30,2020-07-31,2020-08-31,2020-09-30,2020-10-31,2020-11-30,2020-12-31,2021-01-31,2021-02-28,2021-03-31,2021-04-30,2021-05-31,2021-06-30,2021-07-31,2021-08-31,2021-09-30,2021-10-31,2021-11-30,2021-12-31,2022-01-31,2022-02-28,2022-03-31,2022-04-30,2022-05-31,2022-06-30


In [313]:
start_date = data['Fecha comienzo'].min()
end_date = data['Fecha fin'].max()

date_range = pd.date_range(start=start_date, end=end_date, freq='M')
calendario = pd.DataFrame(date_range, columns=['Fecha'])
calendario['Fecha'] = calendario['Fecha'].dt.strftime('%Y-%m')
print(calendario['Fecha'].dtype)
calendario.set_index('Fecha', inplace=True)
calendario=calendario.T
calendario

object


Fecha,2017-01,2017-02,2017-03,2017-04,2017-05,2017-06,2017-07,2017-08,2017-09,2017-10,2017-11,2017-12,2018-01,2018-02,2018-03,2018-04,2018-05,2018-06,2018-07,2018-08,2018-09,2018-10,2018-11,2018-12,2019-01,2019-02,2019-03,2019-04,2019-05,2019-06,2019-07,2019-08,2019-09,2019-10,2019-11,2019-12,2020-01,2020-02,2020-03,2020-04,2020-05,2020-06,2020-07,2020-08,2020-09,2020-10,2020-11,2020-12,2021-01,2021-02,2021-03,2021-04,2021-05,2021-06,2021-07,2021-08,2021-09,2021-10,2021-11,2021-12,2022-01,2022-02,2022-03,2022-04,2022-05,2022-06


In [314]:
calendario1 = calendario.copy()
calendario.insert(loc=0, column='Id', value=data['Id'])
calendario.sample(2)

Fecha,Id,2017-01,2017-02,2017-03,2017-04,2017-05,2017-06,2017-07,2017-08,2017-09,2017-10,2017-11,2017-12,2018-01,2018-02,2018-03,2018-04,2018-05,2018-06,2018-07,2018-08,2018-09,2018-10,2018-11,2018-12,2019-01,2019-02,2019-03,2019-04,2019-05,2019-06,2019-07,2019-08,2019-09,2019-10,2019-11,2019-12,2020-01,2020-02,2020-03,2020-04,2020-05,2020-06,2020-07,2020-08,2020-09,2020-10,2020-11,2020-12,2021-01,2021-02,2021-03,2021-04,2021-05,2021-06,2021-07,2021-08,2021-09,2021-10,2021-11,2021-12,2022-01,2022-02,2022-03,2022-04,2022-05,2022-06
20,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
22,12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [315]:
calendario.set_index('Id', inplace=True)
calendario.sample(2)

Fecha,2017-01,2017-02,2017-03,2017-04,2017-05,2017-06,2017-07,2017-08,2017-09,2017-10,2017-11,2017-12,2018-01,2018-02,2018-03,2018-04,2018-05,2018-06,2018-07,2018-08,2018-09,2018-10,2018-11,2018-12,2019-01,2019-02,2019-03,2019-04,2019-05,2019-06,2019-07,2019-08,2019-09,2019-10,2019-11,2019-12,2020-01,2020-02,2020-03,2020-04,2020-05,2020-06,2020-07,2020-08,2020-09,2020-10,2020-11,2020-12,2021-01,2021-02,2021-03,2021-04,2021-05,2021-06,2021-07,2021-08,2021-09,2021-10,2021-11,2021-12,2022-01,2022-02,2022-03,2022-04,2022-05,2022-06
Id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
14,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [316]:
data2 = data.copy()
data2['Fecha comienzo'] = data2['Fecha comienzo'].dt.strftime('%Y-%m')
data2['Fecha fin'] = data2['Fecha fin'].dt.strftime('%Y-%m')
selected_row = data2.loc[data2['Id'] == 4]

selected_row

,Id,Nombre,Apellido,Area,Fecha comienzo,Fecha fin,Nivel,Grupo recruitment,Tiempo recruitment,Rango Salarial,RS competencia,Manager,Realizo Cursos,pesitos,Nombre del Manager,Apellido del Manager,Mes_comienzo,Mes_fin,Meses_trabajados
0,4,Sebastian,González,Legales,2018-05,2021-05,N2,Grupo A,4,Rango 2,Rango 2,3,SI,200001 - 250000,José,Feligres,2018-05,2021-05,36


In [317]:

# Iterate over all columns in calendario
for column_name in calendario.columns:
    # Check if the value in 'Fecha comienzo' of selected_row matches the current column name
    value = 1 if selected_row['Fecha comienzo'].iloc[0] == column_name else 0

    # Update the value in the current column of calendario for the row with index 4
    calendario.at[4, column_name] = value

    # Break the iteration if the value is 1
    if value == 1:
        break

# Display the updated calendario DataFrame
calendario.head(3)



Fecha,2017-01,2017-02,2017-03,2017-04,2017-05,2017-06,2017-07,2017-08,2017-09,2017-10,2017-11,2017-12,2018-01,2018-02,2018-03,2018-04,2018-05,2018-06,2018-07,2018-08,2018-09,2018-10,2018-11,2018-12,2019-01,2019-02,2019-03,2019-04,2019-05,2019-06,2019-07,2019-08,2019-09,2019-10,2019-11,2019-12,2020-01,2020-02,2020-03,2020-04,2020-05,2020-06,2020-07,2020-08,2020-09,2020-10,2020-11,2020-12,2021-01,2021-02,2021-03,2021-04,2021-05,2021-06,2021-07,2021-08,2021-09,2021-10,2021-11,2021-12,2022-01,2022-02,2022-03,2022-04,2022-05,2022-06
Id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [318]:
data1 = data.copy()
data1['Fecha comienzo'] = data1['Fecha comienzo'].dt.strftime('%Y-%m')
data1['Fecha fin'] = data1['Fecha fin'].dt.strftime('%Y-%m')
selected_row = data1.loc[data1['Id'] == 4]

selected_row


# Flag to track the column where the value was 1
column_start = 0

# Iterate over all columns in calendario
for column_name in calendario.columns[column_start:]:
    # Check if the value in 'Fecha comienzo' of selected_row matches the current column name
    value = 1 if selected_row['Fecha comienzo'].iloc[0] == column_name else 0

    # Update the value in the current column of calendario for the row with index 4
    calendario.at[4, column_name] = value

    # Break the iteration if the value is 1
    if value == 1:
        column_start = calendario.columns.get_loc(column_name) + 1
        break

# If the value was 1, iterate from the next column
for column_name in calendario.columns[column_start:]:
    # Check the 'Fecha fin' condition
    value = 0 if selected_row['Fecha fin'].iloc[0] == column_name else 1

    # Update the value in the current column of calendario for the row with index 4
    calendario.at[4, column_name] = value

    # Break the iteration if the value is 0
    if value == 0:
        break

# After the second iteration, check if 'Fecha fin' is the same as the column name and fill with 1
if selected_row['Fecha fin'].iloc[0] == column_name:
    calendario.at[4, column_name] = 1

# Display the updated calendario DataFrame
pd.set_option('display.max_columns', None)
calendario.head(3)






Fecha,2017-01,2017-02,2017-03,2017-04,2017-05,2017-06,2017-07,2017-08,2017-09,2017-10,2017-11,2017-12,2018-01,2018-02,2018-03,2018-04,2018-05,2018-06,2018-07,2018-08,2018-09,2018-10,2018-11,2018-12,2019-01,2019-02,2019-03,2019-04,2019-05,2019-06,2019-07,2019-08,2019-09,2019-10,2019-11,2019-12,2020-01,2020-02,2020-03,2020-04,2020-05,2020-06,2020-07,2020-08,2020-09,2020-10,2020-11,2020-12,2021-01,2021-02,2021-03,2021-04,2021-05,2021-06,2021-07,2021-08,2021-09,2021-10,2021-11,2021-12,2022-01,2022-02,2022-03,2022-04,2022-05,2022-06
Id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [319]:
# Assuming 'data' is your original DataFrame and 'cacalendario' is your target DataFrame
data1 = data.copy()
data1['Fecha comienzo'] = data1['Fecha comienzo'].dt.strftime('%Y-%m')
data1['Fecha fin'] = data1['Fecha fin'].dt.strftime('%Y-%m')

# Iterate over each row in data1
for index, row in data1.iterrows():
    selected_row = row

    calendario_row_index = selected_row['Id']
    column_start = 0

    # Iterate over all columns in cacalendario
    for column_name in calendario.columns[column_start:]:
        # Check if the value in 'Fecha comienzo' of selected_row matches the current column name
        value = 1 if selected_row['Fecha comienzo'] == column_name else 0

        # Update the value in the current column of cacalendario for the row with index cacalendario_row_index
        calendario.at[calendario_row_index, column_name] = value

        # Break the iteration if the value is 1
        if value == 1:
            column_start = calendario.columns.get_loc(column_name) + 1
            break

    # If the value was 1, iterate from the next column
    for column_name in calendario.columns[column_start:]:
        # Check the 'Fecha fin' condition
        value = 0 if selected_row['Fecha fin'] == column_name else 1

        # Update the value in the current column of cacalendario for the row with index cacalendario_row_index
        calendario.at[calendario_row_index, column_name] = value

        # Break the iteration if the value is 0
        if value == 0:
            # Calculate the starting column index for the fill operation
            fill_start_index = calendario.columns.get_loc(column_name) + 1
            # Fill all the next columns with 0
            calendario.loc[calendario_row_index, calendario.columns[fill_start_index:]] = 0
            break


# Convert all columns in cacalendario to integers
calendario = calendario.astype(int)

# Display the updated cacalendario DataFrame
pd.set_option('display.max_columns', None)
calendario


Fecha,2017-01,2017-02,2017-03,2017-04,2017-05,2017-06,2017-07,2017-08,2017-09,2017-10,2017-11,2017-12,2018-01,2018-02,2018-03,2018-04,2018-05,2018-06,2018-07,2018-08,2018-09,2018-10,2018-11,2018-12,2019-01,2019-02,2019-03,2019-04,2019-05,2019-06,2019-07,2019-08,2019-09,2019-10,2019-11,2019-12,2020-01,2020-02,2020-03,2020-04,2020-05,2020-06,2020-07,2020-08,2020-09,2020-10,2020-11,2020-12,2021-01,2021-02,2021-03,2021-04,2021-05,2021-06,2021-07,2021-08,2021-09,2021-10,2021-11,2021-12,2022-01,2022-02,2022-03,2022-04,2022-05,2022-06
Id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
7,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
8,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0
10,0,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
13,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,0,0,0,0
15,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,0,0,0,0,0,0,0
17,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,0
19,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
24,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,0,0,0,0,0,0


## 6.1 Aqui hago una suma de cuantos trabajadores habia por mes en base a los datos que nos pasaron

In [320]:
Trabajadores = calendario.sum().to_frame()
Trabajadores.rename(columns={0: 'n_trabajadores'}, inplace=True)
Trabajadores.T

Fecha,2017-01,2017-02,2017-03,2017-04,2017-05,2017-06,2017-07,2017-08,2017-09,2017-10,2017-11,2017-12,2018-01,2018-02,2018-03,2018-04,2018-05,2018-06,2018-07,2018-08,2018-09,2018-10,2018-11,2018-12,2019-01,2019-02,2019-03,2019-04,2019-05,2019-06,2019-07,2019-08,2019-09,2019-10,2019-11,2019-12,2020-01,2020-02,2020-03,2020-04,2020-05,2020-06,2020-07,2020-08,2020-09,2020-10,2020-11,2020-12,2021-01,2021-02,2021-03,2021-04,2021-05,2021-06,2021-07,2021-08,2021-09,2021-10,2021-11,2021-12,2022-01,2022-02,2022-03,2022-04,2022-05,2022-06
n_trabajadores,1,1,3,4,5,5,5,5,5,5,5,5,5,6,7,7,9,10,10,10,10,11,11,11,12,13,13,13,13,14,15,16,16,16,16,15,17,17,16,15,15,16,18,18,18,18,17,15,15,15,15,15,14,12,13,13,14,14,12,9,10,8,7,5,3,1


## 6.1 Saldo de altas bajas por mes en base a los datos que nos brindaron. Podrian brindarnos todas las altas y bajas del periodo si quieren un mejor analisis. Gracias, 🥲

In [321]:
Trabajadores['relacion_altasBajas'] = Trabajadores['n_trabajadores'].diff()
Trabajadores.T

Fecha,2017-01,2017-02,2017-03,2017-04,2017-05,2017-06,2017-07,2017-08,2017-09,2017-10,2017-11,2017-12,2018-01,2018-02,2018-03,2018-04,2018-05,2018-06,2018-07,2018-08,2018-09,2018-10,2018-11,2018-12,2019-01,2019-02,2019-03,2019-04,2019-05,2019-06,2019-07,2019-08,2019-09,2019-10,2019-11,2019-12,2020-01,2020-02,2020-03,2020-04,2020-05,2020-06,2020-07,2020-08,2020-09,2020-10,2020-11,2020-12,2021-01,2021-02,2021-03,2021-04,2021-05,2021-06,2021-07,2021-08,2021-09,2021-10,2021-11,2021-12,2022-01,2022-02,2022-03,2022-04,2022-05,2022-06
n_trabajadores,1.0,1.0,3.0,4.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,6.0,7.0,7.0,9.0,10.0,10.0,10.0,10.0,11.0,11.0,11.0,12.0,13.0,13.0,13.0,13.0,14.0,15.0,16.0,16.0,16.0,16.0,15.0,17.0,17.0,16.0,15.0,15.0,16.0,18.0,18.0,18.0,18.0,17.0,15.0,15.0,15.0,15.0,15.0,14.0,12.0,13.0,13.0,14.0,14.0,12.0,9.0,10.0,8.0,7.0,5.0,3.0,1.0
relacion_altasBajas,NaN,0.0,2.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,2.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,-1.0,2.0,0.0,-1.0,-1.0,0.0,1.0,2.0,0.0,0.0,0.0,-1.0,-2.0,0.0,0.0,0.0,0.0,-1.0,-2.0,1.0,0.0,1.0,0.0,-2.0,-3.0,1.0,-2.0,-1.0,-2.0,-2.0,-2.0


## 6.2 Aqui me puse a comparar las altas y las bajas para integrarlas a mi calendario 🚨

In [322]:
altas = data['Fecha comienzo'].dt.strftime('%Y-%m')
altas.sort_values()

2     2017-01
3     2017-03
12    2017-03
15    2017-04
20    2017-05
26    2017-11
7     2018-02
11    2018-03
0     2018-05
18    2018-05
19    2018-06
5     2018-10
16    2018-12
8     2019-01
23    2019-02
13    2019-04
9     2019-06
14    2019-07
24    2019-08
10    2020-01
6     2020-01
4     2020-06
22    2020-07
1     2020-07
21    2021-01
28    2021-03
17    2021-07
25    2021-09
29    2021-09
27    2022-01
Name: Fecha comienzo, dtype: object

In [323]:
bajas = data['Fecha fin'].dt.strftime('%Y-%m')
bajas.sort_values()

20    2017-11
11    2018-12
19    2019-04
15    2019-12
23    2020-03
24    2020-04
3     2020-11
22    2020-12
26    2020-12
7     2021-01
1     2021-03
0     2021-05
16    2021-06
2     2021-06
18    2021-09
5     2021-11
21    2021-11
8     2021-12
12    2021-12
28    2021-12
4     2022-02
17    2022-02
13    2022-03
14    2022-04
9     2022-04
25    2022-05
6     2022-05
10    2022-06
29    2022-06
27    2022-07
Name: Fecha fin, dtype: object

In [324]:
Trabajadores['n_altas'] = 0
altas_counts = altas.value_counts()
for date, count in altas_counts.items():
    Trabajadores.loc[Trabajadores.index == date, 'n_altas'] = count
Trabajadores['n_bajas'] = 0
bajas_counts = bajas.value_counts()
for date, count in bajas_counts.items():
    Trabajadores.loc[Trabajadores.index == date, 'n_bajas'] = count
Trabajadores.T

Fecha,2017-01,2017-02,2017-03,2017-04,2017-05,2017-06,2017-07,2017-08,2017-09,2017-10,2017-11,2017-12,2018-01,2018-02,2018-03,2018-04,2018-05,2018-06,2018-07,2018-08,2018-09,2018-10,2018-11,2018-12,2019-01,2019-02,2019-03,2019-04,2019-05,2019-06,2019-07,2019-08,2019-09,2019-10,2019-11,2019-12,2020-01,2020-02,2020-03,2020-04,2020-05,2020-06,2020-07,2020-08,2020-09,2020-10,2020-11,2020-12,2021-01,2021-02,2021-03,2021-04,2021-05,2021-06,2021-07,2021-08,2021-09,2021-10,2021-11,2021-12,2022-01,2022-02,2022-03,2022-04,2022-05,2022-06
n_trabajadores,1.0,1.0,3.0,4.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,6.0,7.0,7.0,9.0,10.0,10.0,10.0,10.0,11.0,11.0,11.0,12.0,13.0,13.0,13.0,13.0,14.0,15.0,16.0,16.0,16.0,16.0,15.0,17.0,17.0,16.0,15.0,15.0,16.0,18.0,18.0,18.0,18.0,17.0,15.0,15.0,15.0,15.0,15.0,14.0,12.0,13.0,13.0,14.0,14.0,12.0,9.0,10.0,8.0,7.0,5.0,3.0,1.0
relacion_altasBajas,NaN,0.0,2.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,2.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,-1.0,2.0,0.0,-1.0,-1.0,0.0,1.0,2.0,0.0,0.0,0.0,-1.0,-2.0,0.0,0.0,0.0,0.0,-1.0,-2.0,1.0,0.0,1.0,0.0,-2.0,-3.0,1.0,-2.0,-1.0,-2.0,-2.0,-2.0
n_altas,1.0,0.0,2.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,2.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,1.0,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,1.0,2.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,2.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
n_bajas,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,2.0,1.0,0.0,1.0,0.0,1.0,2.0,0.0,0.0,1.0,0.0,2.0,3.0,0.0,2.0,1.0,2.0,2.0,2.0


## 6.2.1 La relacion altas bajas parece verse influenciada por la pandemia. Este analisis tendria mas sentido si tenemos la cantidad de altas totales 👮

In [325]:
altas_anual = data['Fecha comienzo'].dt.strftime('%Y').value_counts()
bajas_anual = data['Fecha fin'].dt.strftime('%Y').value_counts()

# Convert each Series to a DataFrame and reset the index
altas_anual_df = altas_anual.to_frame(name='altas').reset_index()
bajas_anual_df = bajas_anual.to_frame(name='bajas').reset_index()

# Rename the index column to 'Año'
altas_anual_df.rename(columns={'index': 'Año'}, inplace=True)
bajas_anual_df.rename(columns={'index': 'Año'}, inplace=True)

# Merge the two DataFrames on 'Año', using a left join to include all years from altas_anual
relacion_altas_bajas_anual = pd.merge(altas_anual_df, bajas_anual_df, on='Año', how='left')

# Calculate the difference between altas and bajas and add it as a new column
relacion_altas_bajas_anual['dif'] = relacion_altas_bajas_anual['altas'] - relacion_altas_bajas_anual['bajas']

# Append a final row with the total counts of altas and bajas
total_altas = altas_anual.sum()
total_bajas = bajas_anual.sum()
total_dif = total_altas - total_bajas
relacion_altas_bajas_anual = relacion_altas_bajas_anual.append({'Año': 'Total', 'altas': total_altas, 'bajas': total_bajas, 'dif': total_dif}, ignore_index=True)

relacion_altas_bajas_anual.sort_values('Año')


<ipython-input-325-41f87ab75f05>:22: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  relacion_altas_bajas_anual = relacion_altas_bajas_anual.append({'Año': 'Total', 'altas': total_altas, 'bajas': total_bajas, 'dif': total_dif}, ignore_index=True)


,Año,altas,bajas,dif
1,2017,6,1,5
0,2018,7,1,6
2,2019,6,2,4
3,2020,5,5,0
4,2021,5,11,-6
5,2022,1,10,-9
6,Total,30,30,0


# 7. Gracias !! 🤗

![](https://media.giphy.com/media/3oFzmbQom9DGp7OhZS/giphy.gif)